In this notebook we will go through each accident registered in our dataset and look at the snow informations retrieved from the maps dataset to relate both of them.

In [1]:
import numpy as np
import pandas as pd
import folium
import branca
import json
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
# Load avalanche accidents data
df_accidents = pd.read_excel('../data/accidents/raw_accidents.xlsx')
coordinates = pd.read_excel('../data/accidents/coordinates.xlsx')
df_accidents = df_accidents.assign(Longitude = coordinates.Longitude, Latitude = coordinates.Latitude)
df_accidents = df_accidents.drop(['Aspect', 'Activity', 'Danger level', 'caught', 'buried', 'killed'], axis=1)
df_accidents.head()

,Date,Canton,Starting zone,Elevation,Latitude,Longitude
0,1997-10-26,BE,Kandersteg,3420,46.469813,7.728972
1,1997-12-19,GR,St. Moritz,2560,46.502976,9.772840
2,1997-12-21,UR,Andermatt,2940,46.602754,8.612660
3,1997-12-29,VS,Bagnes,2435,46.034050,7.199696
4,1998-01-04,UR,Andermatt,2400,46.617105,8.616889


Our main interest here are the places and dates.